In [1]:
import pandas as pd
import re

In [2]:
from bs4 import BeautifulSoup

with open('the2017report.html', 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'html.parser')

In [3]:
print(soup.prettify().split('#231F20;">')[5][:15])
print(len(soup.prettify().split('#231F20;">')[5].split('<p class="s1'))-1)


    CALIFORNIA
190


In [4]:
calilen = len(soup.prettify().split('#231F20;">')[5].split('<p class="s1'))-1

**test round**

In [5]:
for i in range(15,16):
    textbox = soup.prettify().split('#231F20;">')[5].split('<p class="s1')[i].split('</p>\n')

In [6]:
textbox

['" style="padding-left: 14pt;text-indent: 0pt;line-height: 83%;text-align: left;">\n   Human Designs Prosthetic &amp; Orthotic Laboratory Inc.\n   <span class="s2">\n    ❹\n   </span>\n   <span class="p">\n    49 E Foothill Blvd\n   </span>\n  ',
 '  <p style="padding-left: 14pt;text-indent: 0pt;text-align: left;">\n   Arcadia\n   <span class="s6">\n    ,\n   </span>\n   CA 91006-2870\n  ',
 '  <p style="padding-left: 13pt;text-indent: 0pt;text-align: left;">\n   (866) 988-2414\n  ',
 '  <p style="padding-left: 13pt;text-indent: 0pt;text-align: left;">\n   (626) 445-7797\n  ',
 '  <p style="padding-left: 13pt;text-indent: 0pt;text-align: left;">\n   Fax (626) 445-7873\n  ',
 '  <p style="padding-left: 14pt;text-indent: 0pt;text-align: left;">\n   <a class="a" href="mailto:info@humandesigns.com" target="_blank">\n    info@humandesigns.\n   </a>\n   <a class="a" href="http://www.humandesigns.com/" target="_blank">\n    com www.humandesigns.\n   </a>\n   com\n   <span class="s5">\n    Er

In [9]:
def extract_all_content_between_newlines(text):
    contents = []
    start_index = 0

    while True:
        start_index = text.find('\n', start_index)
        if start_index == -1:
            break

        end_index = text.find('\n', start_index + 1)
        if end_index == -1:
            break

        content_between_newlines = text[start_index + 1:end_index].strip()
        if content_between_newlines:
            contents.append(content_between_newlines)
        
        start_index = end_index  # Set the start_index as the end_index

    combined_contents = " ".join(contents)

    return combined_contents

# Example usage:
line_of_text = textbox[5]
combined_content = extract_all_content_between_newlines(line_of_text)

if combined_content:
    print("Combined extracted content:\n", combined_content)
else:
    print("No content found between newlines in the line of text.")

Combined extracted content:
 <a class="a" href="mailto:info@humandesigns.com" target="_blank"> info@humandesigns. </a> <a class="a" href="http://www.humandesigns.com/" target="_blank"> com www.humandesigns. </a> com <span class="s5"> Eric A. Ferraco, CPO, President </span>


In [10]:
combined_content

'<a class="a" href="mailto:info@humandesigns.com" target="_blank"> info@humandesigns. </a> <a class="a" href="http://www.humandesigns.com/" target="_blank"> com www.humandesigns. </a> com <span class="s5"> Eric A. Ferraco, CPO, President </span>'

In [11]:
def process_data(data):
    def extract_email_url_and_name(text):
        url_pattern = r'href="(.*?)"'
        name_pattern = r'<span.*?>(.*?)<\/span>'
        
        url_matches = re.findall(url_pattern, text)
        name_match = re.search(name_pattern, text)
        
        urls = url_matches if url_matches else []
        name = name_match.group(1).strip() if name_match else ''
        
        return urls, name
    
    urls, name = extract_email_url_and_name(data)
    
    return urls, name

In [12]:
proce = process_data(combined_content)
proce

(['mailto:info@humandesigns.com', 'http://www.humandesigns.com/'],
 'Eric A. Ferraco, CPO, President')

In [13]:
def remove_closing_angle_brackets(text):
    cleaned_text = re.sub(r'</[^>]+>|<br/>|<[^>]+>|</span>|</h2>', '', text)
    return cleaned_text

In [14]:
remove_angle = remove_closing_angle_brackets(combined_content)
remove_angle

' info@humandesigns.   com www.humandesigns.  com  Eric A. Ferraco, CPO, President '

In [15]:
def flatten_list(lst):
    result = []
    for item in lst:
        if isinstance(item, (list, tuple)):
            result.extend(flatten_list(item))
        else:
            result.append(item)
    return result

**test**

In [16]:
res = []
for j in range(2,3):
    cl = soup.prettify().split('#231F20;">')[5].split('<p class="s1')[j].split('</p>\n')
    row = [extract_all_content_between_newlines(c) for c in cl]
    row_c1 = [remove_closing_angle_brackets(c1) for c1 in row]
    res.append(row_c1)

In [17]:
res

[['Sierra Prosthetics -Orthotics  ❹ ',
  '3227 Professional Dr',
  'Auburn  ,  CA 95602',
  '(530) 885-9922',
  ' sierrapo@gv.net ',
  '',
  'BAKERSFIELD   TUCSON   Alicia Lenhart, RFOM  ',
  '',
  '',
  '']]

In [19]:
row

['Sierra Prosthetics -Orthotics <span class="s2"> ❹ </span>',
 '3227 Professional Dr',
 'Auburn <span class="s6"> , </span> CA 95602',
 '(530) 885-9922',
 '<a href="mailto:sierrapo@gv.net"> sierrapo@gv.net </a>',
 '<br/>',
 'BAKERSFIELD </h2> <h2 style="padding-top: 6pt;padding-left: 14pt;text-indent: 0pt;line-height: 4pt;text-align: left;"> TUCSON </h2> <p class="s5" style="padding-top: 1pt;padding-left: 14pt;text-indent: 0pt;line-height: 10pt;text-align: left;"> Alicia Lenhart, RFOM <u> </u>',
 '<br/>',
 '',
 '']

In [20]:
processed_values = [
    process_data(value) if value.startswith('<a') else value
    for value in row
]
processed_values

['Sierra Prosthetics -Orthotics <span class="s2"> ❹ </span>',
 '3227 Professional Dr',
 'Auburn <span class="s6"> , </span> CA 95602',
 '(530) 885-9922',
 (['mailto:sierrapo@gv.net'], ''),
 '<br/>',
 'BAKERSFIELD </h2> <h2 style="padding-top: 6pt;padding-left: 14pt;text-indent: 0pt;line-height: 4pt;text-align: left;"> TUCSON </h2> <p class="s5" style="padding-top: 1pt;padding-left: 14pt;text-indent: 0pt;line-height: 10pt;text-align: left;"> Alicia Lenhart, RFOM <u> </u>',
 '<br/>',
 '',
 '']

In [21]:
flattened_list = flatten_list(processed_values)
flattened_list

['Sierra Prosthetics -Orthotics <span class="s2"> ❹ </span>',
 '3227 Professional Dr',
 'Auburn <span class="s6"> , </span> CA 95602',
 '(530) 885-9922',
 'mailto:sierrapo@gv.net',
 '',
 '<br/>',
 'BAKERSFIELD </h2> <h2 style="padding-top: 6pt;padding-left: 14pt;text-indent: 0pt;line-height: 4pt;text-align: left;"> TUCSON </h2> <p class="s5" style="padding-top: 1pt;padding-left: 14pt;text-indent: 0pt;line-height: 10pt;text-align: left;"> Alicia Lenhart, RFOM <u> </u>',
 '<br/>',
 '',
 '']

In [22]:
row_c1 = [remove_closing_angle_brackets(c1) for c1 in flattened_list]
row_c1

['Sierra Prosthetics -Orthotics  ❹ ',
 '3227 Professional Dr',
 'Auburn  ,  CA 95602',
 '(530) 885-9922',
 'mailto:sierrapo@gv.net',
 '',
 '',
 'BAKERSFIELD   TUCSON   Alicia Lenhart, RFOM  ',
 '',
 '',
 '']

In [25]:
res = []
for j in range(2,3):
    cl = soup.prettify().split('#231F20;">')[5].split('<p class="s1')[j].split('</p>\n')
    row = [extract_all_content_between_newlines(c) for c in cl]
    processed_values = [process_data(value) if value.startswith('<a') else value for value in row]
    flattened_list = flatten_list(processed_values)
    row_c1 = [remove_closing_angle_brackets(c1) for c1 in flattened_list]
    res.append(row_c1)

In [26]:
res

[['Sierra Prosthetics -Orthotics  ❹ ',
  '3227 Professional Dr',
  'Auburn  ,  CA 95602',
  '(530) 885-9922',
  'mailto:sierrapo@gv.net',
  '',
  '',
  'BAKERSFIELD   TUCSON   Alicia Lenhart, RFOM  ',
  '',
  '',
  '']]

**loop**

In [27]:
res = []
for j in range(calilen):
    cl = soup.prettify().split('#231F20;">')[5].split('<p class="s1')[j].split('</p>\n')
    row = [extract_all_content_between_newlines(c) for c in cl]
    processed_values = [process_data(value) if value.startswith('<a') else value for value in row]
    flattened_list = flatten_list(processed_values)
    row_c1 = [remove_closing_angle_brackets(c1) for c1 in flattened_list]
    res.append(row_c1)

In [28]:
len(res)

190

In [29]:
df = pd.DataFrame(res)

In [30]:
# df

In [31]:
df.to_excel('aopa_cali.xlsx',index=False)

**trouble-shooting**

In [19]:
for i in range(7,8):
    textbox = soup.prettify().split('#231F20;">')[5].split('<p class="s1')[i].split('</p>\n')

In [20]:
textbox

['" style="padding-top: 10pt;padding-left: 13pt;text-indent: 0pt;line-height: 86%;text-align: left;">\n   North County Prosthetics &amp; Orthotics\n   <span class="s2">\n    ❹\n   </span>\n  ',
 '  <p style="padding-left: 13pt;text-indent: 0pt;line-height: 8pt;text-align: left;">\n   200 Station Way, Ste C\n  ',
 '  <p style="padding-left: 13pt;text-indent: 0pt;line-height: 107%;text-align: left;">\n   Arroyo Grande, CA 93420 (805) 481-9666\n  ',
 '  <p style="padding-left: 13pt;text-indent: 0pt;text-align: left;">\n   Fax (805) 466-9504\n  ',
 '  <p style="padding-left: 13pt;text-indent: 0pt;text-align: left;">\n   <a class="a" href="http://www.northcountyprosthetics.com/" target="_blank">\n    ncpo@sbcglobal.net www.northcountyprosthetics.\n   </a>\n   com\n   <span class="s5">\n    Jamie E. Hollingsead, COO, Owner John D. Hollingsead, CPO, Owner Timothy Bulgarelli, CPO\n   </span>\n  ',
 '  <p class="s5" style="padding-left: 13pt;text-indent: 0pt;line-height: 7pt;text-align: left;">

In [23]:
line_of_text = textbox[4]
combined_content = extract_all_content_between_newlines(line_of_text)

if combined_content:
    print("Combined extracted content:\n", combined_content)
else:
    print("No content found between newlines in the line of text.")

Combined extracted content:
 <a class="a" href="http://www.northcountyprosthetics.com/" target="_blank"> ncpo@sbcglobal.net www.northcountyprosthetics. </a> com <span class="s5"> Jamie E. Hollingsead, COO, Owner John D. Hollingsead, CPO, Owner Timothy Bulgarelli, CPO </span>


In [24]:
combined_content

'<a class="a" href="http://www.northcountyprosthetics.com/" target="_blank"> ncpo@sbcglobal.net www.northcountyprosthetics. </a> com <span class="s5"> Jamie E. Hollingsead, COO, Owner John D. Hollingsead, CPO, Owner Timothy Bulgarelli, CPO </span>'

**html patterns**

In [29]:
# soup.prettify().split('#231F20;">')[5].split('<p class="s1')

In [88]:
# soup.prettify().split('#231F20;">') #state names
# soup.prettify().split('<h2 style="') #city names
# soup.prettify().split('<p class="s1') #clinic names
# soup.prettify().split('</p>\n')[1] #each line

In [89]:
contents[6800:8000]

'dy {vertical-align: top; overflow: visible; }\n</style></head><body><p style="padding-top: 9pt;padding-left: 14pt;text-indent: 0pt;text-align: left;"><span class="h1" style=" background-color: #231F20;">      ALASKA    </span></p><p style="text-indent: 0pt;text-align: left;"><br/></p><h2 style="padding-bottom: 2pt;padding-left: 14pt;text-indent: 0pt;text-align: left;">ANCHORAGE</h2><p style="padding-left: 14pt;text-indent: 0pt;line-height: 1pt;text-align: left;"/><p class="s1" style="padding-top: 3pt;padding-left: 14pt;text-indent: 0pt;line-height: 16pt;text-align: left;">Northern Orthopedics Inc. <span class="s2">C</span></p><p style="padding-left: 14pt;text-indent: 0pt;line-height: 8pt;text-align: left;">3400 LaTouche St, Ste 100</p><p class="s3" style="padding-left: 14pt;text-indent: 0pt;text-align: left;">Anchorage, <span class="s4">AK 99508-4208</span></p><p style="padding-left: 14pt;text-indent: 0pt;text-align: left;">(907) 561-4385</p><p style="padding-left: 14pt;text-indent: 0

**make it into df**

In [55]:
# source = urllib.request.urlopen('http://www.gcoins.net/en/catalog/view/45518').read()
# soup = BeautifulSoup(source, "html.parser")
# table = soup.find('table', attrs={'class':'subs noBorders evenRows'})

In [56]:
# table_rows = table.find_all('tr')

# res = []
# for tr in table_rows:
#     td = tr.find_all('td')
#     row = [tr.text.strip() for tr in td if tr.text.strip()]
#     if row:
#         res.append(row)

In [ ]:
# l = []
# for tr in table_rows:
#     td = tr.find_all('td')
#     row = [tr.text for tr in td]
#     l.append(row)
# pd.DataFrame(l, columns=["A", "B", ...])